In [ ]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cuda:0


In [ ]:
cd /content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images

/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images


In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d timesler/facenet-pytorch-vggface2

 92% 103M/112M [00:02<00:00, 38.9MB/s] 
100% 112M/112M [00:02<00:00, 49.4MB/s]


In [ ]:
!cd /content/
!mkdir facenet-pytorch-vggface2
!unzip facenet-pytorch-vggface2.zip
!mv *.whl facenet-pytorch-vggface2/
!mv *.pth facenet-pytorch-vggface2/
!pip install /content/facenet-pytorch-vggface2/facenet_pytorch-2.0.0-py3-none-any.whl

Archive:  facenet-pytorch-vggface2.zip
  inflating: 20180402-114759-vggface2-features.pth  
  inflating: 20180402-114759-vggface2-logits.pth  
  inflating: facenet_pytorch-1.0.1-py3-none-any.whl  
  inflating: facenet_pytorch-2.0.0-py3-none-any.whl  
  inflating: facenet_pytorch-2.0.1-py3-none-any.whl  
  inflating: facenet_pytorch-2.1.0-py3-none-any.whl  
  inflating: facenet_pytorch-2.1.1-py3-none-any.whl  
  inflating: facenet_pytorch-2.2.7-py3-none-any.whl  
  inflating: facenet_pytorch-2.2.9-py3-none-any.whl  
Processing ./facenet-pytorch-vggface2/facenet_pytorch-2.0.0-py3-none-any.whl


In [ ]:
from facenet_pytorch.models.inception_resnet_v1 import get_torch_home
torch_home = get_torch_home()

# Copy model checkpoints to torch cache so they are loaded automatically by the package
!mkdir -p $torch_home/checkpoints/
!cp /content/facenet-pytorch-vggface2/20180402-114759-vggface2-logits.pth $torch_home/checkpoints/vggface2_DG3kwML46X.pt
!cp /content/facenet-pytorch-vggface2/20180402-114759-vggface2-features.pth $torch_home/checkpoints/vggface2_G5aNV2VSMn.pt

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_sample_videos/'
BATCH_SIZE = 1
SCALE = 0.25
N_FRAMES = None # The number of frames extracted from each video, 'None' means get all available frames

In [ ]:
!git clone https://github.com/HongguLiu/Deepfake-Detection

Cloning into 'Deepfake-Detection'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 108 (delta 34), reused 6 (delta 2), pack-reused 41
Receiving objects: 100% (108/108), 129.93 MiB | 20.75 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
!pip install pretrainedmodels==0.7.4

     |████████████████████████████████| 61kB 6.0MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60966 sha256=6a26c72262ec0d9890f984cfa420f45f9a1f6c5a1bad2d2798d4563ed53addf3
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [ ]:
import sys
sys.path.insert(0, "/content/Deepfake-Detection/network/")

In [ ]:
sys.path.insert(1,'/content/Deepfake-Detection/')

In [ ]:
!mkdir -p /home/ondyari/.torch/models

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1PDGYCwVv-ANPHEFzbDKo9rPmhAFRxrrM'

output = '/home/ondyari/.torch/models/xception-b5690688.pth'
gdown.download(url,output,quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1PDGYCwVv-ANPHEFzbDKo9rPmhAFRxrrM
To: /home/ondyari/.torch/models/xception-b5690688.pth
91.7MB [00:00, 123MB/s] 


'/home/ondyari/.torch/models/xception-b5690688.pth'

In [ ]:
class DetectionPipeline:
    """Pipeline class for detecting faces in the frames of a video file."""
    
    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):
        """Constructor for DetectionPipeline class.
        
        Keyword Arguments:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            batch_size {int} -- Batch size to use with MTCNN face detector. (default: {32})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Create video reader and find length
        # CAP_PROP_FRA,E_COUNT : 동영상 총 프레임 수 출력
        # https://076923.github.io/posts/Python-opencv-4/
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT)) 

        # Pick 'n_frames' evenly spaced frames to sample

        # linspace() : 1차원 배열 만들기, 수평축 간격 만들기 등에 활용
        # np.linspace(start,stop,num)
        # https://m.blog.naver.com/choi_s_h/221730568009
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        faces = []
        frames = []
        for j in range(v_len):
          #cv2.VideoCapture.grab() : 비디오 파일, 영상 장치로부터 다른 frame 반환, 성공 시 True 리턴
            success = v_cap.grab()
            if j in sample:
                # Load frame
                #cv2.VideoCapture.retrieve() : grab한 frame을 decode해서 반환
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # numpy 배열을 PIL 이미지로 변환
                frame = Image.fromarray(frame)
                
                # Resize frame to desired size
                # PIL.resize() : https://ponyozzang.tistory.com/600
                # frame.size는 (width,height) 반환
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])
                frames.append(frame)

                # When batch is full, detect faces and reset frame list
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    faces.extend(self.detector(frames))
                    frames = []

        v_cap.release()

        return faces

In [ ]:
def process_faces(faces, feature_extractor):
    # Filter out frames without faces
    faces = [f for f in faces if f is not None]
    if len(faces) == 0:
        return None
    faces = torch.cat(faces).to(device)

    # Generate facial feature vectors using a pretrained model
    embeddings = feature_extractor(faces)

    # Calculate centroid for video and distance of each face's feature vector from centroid
    # centroid : 도형의 중심
    #.mean(dim=0) : 첫번째 차원 단위의 평균 , https://curioso365.tistory.com/48
    centroid = embeddings.mean(dim=0)
    # 거리를 구한 후 cpu로 다시 전송하고 numpay 배열로 변환
    x = (embeddings - centroid).norm(dim=1).cpu().numpy()
    
    return x

In [ ]:
# Load face detector
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()
# Load facial recognition model
feature_extractor = InceptionResnetV1(pretrained='vggface2', device=device).eval()
# Define face detection pipeline
detection_pipeline = DetectionPipeline(detector=face_detector, n_frames=N_FRAMES, batch_size=BATCH_SIZE, resize=SCALE)

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/train_sample_videos/'
SAVE_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images/'
folders = os.listdir(TRAIN_DIR)
folders.remove("metadata.json")

In [ ]:
!pip install mtcnn
from mtcnn import MTCNN

     |████████████████████████████████| 2.3MB 14.2MB/s 


In [ ]:
# Get the paths of all train videos
# glob.glob() : 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))

# Get path of metadata.json
metadata_path = TRAIN_DIR + 'metadata.json'

# Get metadata
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

KeyboardInterrupt: ignored

In [ ]:
"""df = pd.DataFrame(columns=['filename', 'distance', 'label'])

with torch.no_grad():
    for path in tqdm(all_train_videos):
        file_name = path.split('/')[-1]

        # Detect all faces occur in the video
        faces = detection_pipeline(path) #detection_pipeline 객체는 callable
        
        # Calculate the distances of all faces' feature vectors to the centroid
        distances = process_faces(faces, feature_extractor)
        if distances is None:
            continue

        for distance in distances:
            row = [
                file_name,
                distance,
                1 if metadata[file_name]['label'] == 'FAKE' else 0
            ]

            # Append a new row at the end of the data frame
            df.loc[len(df)] = row"""

In [ ]:
all_train_images = glob.glob(os.path.join(SAVE_DIR, '*.jpg'))

import json

metadata_path = TRAIN_DIR + 'metadata.json'
with open(metadata_path, 'r') as f :
  metadata = json.load(f)

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train.csv'
SAVE_PATH = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/model/model.pth'
TEST_SIZE = 0.3
RANDOM_STATE = 128
EPOCHS = 200
BATCH_SIZE = 64
LR = 1e-4

In [ ]:
train_df = pd.read_csv(TRAIN_PATH)
r_idx =[i for i in range(train_df.shape[0]-1,-1,-1)]
train_df = pd.DataFrame(train_df,index=r_idx)

In [ ]:
# groupby().count(): https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221578773214
label_count = train_df.groupby('label').count()['filename']
print(label_count)

# Use pos_weight value to overcome imbalanced dataset.
# https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss
pos_weight = torch.ones([1]) * label_count[0]/label_count[1]
print('pos_weight:', pos_weight)

label
0     27476
1    100079
Name: filename, dtype: int64
pos_weight: tensor([0.2745])


In [ ]:
from sklearn.model_selection import train_test_split
# X : data, y : label
X = train_df['distance'].to_numpy()
y = train_df['label'].to_numpy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [ ]:
#X_train = torch.tensor(X_train).to(device).unsqueeze(dim=1).float()
#X_val = torch.tensor(X_val).to(device).unsqueeze(dim=1).float()
#y_train = torch.tensor(y_train).to(device).unsqueeze(dim=1).float()
#y_val = torch.tensor(y_val).to(device).unsqueeze(dim=1).float()

print(X_train)

[0.99945897 0.47975868 0.26516011 ... 0.60652989 0.262925   0.33295554]


In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import os
import random


class myDataset(Dataset):
    def __init__(self, X_train, y_train):
        #fh = open(txt_path, 'r')
        datas = []
        for j in range(len(X_train)):
          datas.append((X_train[j], y_train[j]))
        
        self.datas = datas

    def __getitem__(self, index):
        distance, label = self.datas[index]

        return distance, label
    
    #def getPath(self,index):
    #  return (self.imgs[index])[0]

    def __len__(self):
        return len(self.datas)

In [ ]:
def shuffle_data(X, y):
    assert len(X) == len(y)
    
    p = np.random.permutation(len(X))
    
    return X[p], y[p]

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Thu Jul 15 15:00:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_path2 = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/faceforensics++_models/faceforensics++_models_subset/full/xception/full_raw.p'

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
import argparse
import os
import cv2

from network.models import model_selection
from mesonet import Meso4, MesoInception4
from dataset.transform import xception_default_data_transforms
from dataset.mydataset import MyDataset

def train(output_name='xception_weights', output_order = 1,train=[],train_label = [],val=[],val_label = [],epoches = 20,batch_size = 38,  model_path=None):
  folder_name = 'train' + '_' + str(output_order)
  output_path = os.path.join(model_path,folder_name)
  os.mkdir(output_path)

  torch.backends.cudnn.benchmark = True
  train_dataset = myDataset(X_train = train, y_train = train_label)
  val_dataset = myDataset(X_train = val, y_train = val_label)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
  train_dataset_size = len(train_dataset)
  val_dataset_size = len(val_dataset)
 
  model = model_selection(modelname='xception', num_out_classes=2, dropout=0.5)
  #model = torch.load(model_path2)
  model = model.cuda()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
 
  model = nn.DataParallel(model)
  best_model_wts = model.state_dict()
  best_acc = 0.0
  iteration = 0

  for epoch in tqdm(range(epoches)):
    print('Epoch {}/{}'.format(epoch+1, epoches))
    #print(best_model_wts)
    print('-'*10)
    model.train()
    train_loss = 0.0
    train_corrects = 0.0
    val_loss = 0.0
    val_corrects = 0.0
    for (distance,labels) in train_loader:
      iter_loss = 0.0
      iter_corrects = 0.0
      distance = distance.cuda()
      labels = labels.cuda()
      optimizer.zero_grad()
      outputs = model(distance)
      _, preds = torch.max(outputs.data, 1)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      iter_loss = loss.data.item()
      train_loss += iter_loss
      iter_corrects = torch.sum(preds == labels.data).to(torch.float32)
      train_corrects += iter_corrects
      iteration += 1
      if not (iteration % 20):
        print('iteration {} train loss: {:.4f} Acc: {:.4f}'.format(iteration, iter_loss / batch_size, iter_corrects / batch_size))
    epoch_loss = train_loss / train_dataset_size
    epoch_acc = train_corrects / train_dataset_size
    print('epoch train loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    model.eval()
    with torch.no_grad():
      for (distance, labels) in val_loader:
        distance = distance.cuda()
        labels = labels.cuda()
        outputs = model(distance)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        val_loss += loss.data.item()
        val_corrects += torch.sum(preds == labels.data).to(torch.float32)
      epoch_loss = val_loss / val_dataset_size
      epoch_acc = val_corrects / val_dataset_size
      print('epoch val loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
      if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = model.state_dict()
      scheduler.step()
		#if not (epoch % 40):
    torch.save(model.state_dict(), output_path + '/' + output_name + str(epoch) + ".pkl")
  print('Best val Acc: {:.4f}'.format(best_acc))
  model.load_state_dict(best_model_wts)
  torch.save(model.state_dict(), output_path + '/' + output_name + '_' + "best.pkl")

In [ ]:
SAVE_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/model/firstXceptionRunModel/'

In [ ]:
y_train[0]

1

In [ ]:
train(output_name='xception_weights', output_order = 4,train = X_train, train_label = y_train,val= X_val, val_label = y_val,model_path=SAVE_DIR)

Using dropout 0.5


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1/20
----------


RuntimeError: ignored